In [2]:
import pandas as pd
import numpy as np
import math
import os
import pickle
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML
import csv

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly.colors import DEFAULT_PLOTLY_COLORS

from simulator import simulate as _simulate
from simulator_plotting import *

init_notebook_mode(connected=True)

def simulate(landscape, **kwargs):
    # wrapper function for the c extension's simulate

    # landscape could contain Series or ndarrays. convert them to lists.
    try:
        landscape = [i if isinstance(i, list) else i.tolist() for i in landscape]
    except (TypeError, AttributeError) as err:
        raise TypeError('invalid landscape') from err
    
    return _simulate(landscape, **kwargs)


pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item] for item in pair] for pair in pairs]

In [3]:
def highlight_yes(v):
    if v == 'Yes':
        return 'font-weight: bold'
    else:
        return ''
def highlight_max(s):
    is_max = s == s.max()
    return ['font-style: italic' if v else '' for v in is_max]

def pathway_analysis(landscape, num=100, pretty=False, **kwargs):
    #success_count = 0
    greedy_path = ''
    paths = {}
    times = [] # T_f's
    global_optimum = ''
    local_optima = []
    for i in range(num):
        results = simulate(landscape, **kwargs)
        if not global_optimum:
            global_optimum = results['global_optimum']
            local_optima = ', '.join(results['local_optima'])
        if not greedy_path:
            greedy_path = ','.join(results['greedy_path'])
            paths[greedy_path] = [0, 0]
        actual_path = ','.join(results['actual_path'])
        if actual_path in paths:
            paths[actual_path][0] += 1
        else:
            paths[actual_path] = [1, 0] # (appearances, successful appearances)
        if results['T_f'] != -1:
            paths[actual_path][1] += 1
        times.append(results['T_f'])
    data = []
    total_success_count = sum([sc for c, sc in paths.values()])
    num_paths = sum([c > 0 for c, sc in paths.values()])
    num_successful_paths = sum([sc > 0 for c, sc in paths.values()])
    for path, (count, success_count) in paths.items():
        successful = success_count > 0
        row = OrderedDict()
        row['Pathway'] = path
        row['Successful?'] = 'Yes' if successful else 'No'
        row['Greedy?'] = 'Yes' if path is greedy_path else 'No'
        row['Number of appearances'] = count
        row['Weight'] = success_count / total_success_count if successful else None
        data.append(row)
    df = pd.DataFrame(data)
    s = df.style.applymap(highlight_yes, subset='Successful?').apply(highlight_max, subset='Weight')
    max_weight = df['Weight'].max()
    if not np.isnan(max_weight):
        dominant_path = df.iloc[df['Weight'].idxmax()]['Pathway']
    else:
        dominant_path = ""
    if pretty:
        display(s)
        display(Markdown(f'Global optimum: {global_optimum}'))
        display(Markdown(f'Weight of the dominant successful pathway: {max_weight}'))
        display(Markdown(f'Success rate: {num_successful_paths / num_paths}'))
    else:
        return {
            'paths': df,
            'global_optimum': global_optimum,
            'max_weight': max_weight,
            'success_count': total_success_count,
            #'success_rate': num_successful_paths / num_paths, multiple definitions ughhh
            'success_rate': total_success_count / num,
            'dominant_path': dominant_path,
            'times': times
        }

In [4]:
ZOX_CXM_avg = (pairs[2][0] + pairs[2][1]) / 2
ZOX_CXM_avg

Genotype
0000    1.3705
0001    1.2525
0010    2.0695
0011    2.3105
0100    2.3190
0101    2.0915
0110    2.5280
0111    2.9800
1000    0.7645
1001    1.3745
1010    1.9025
1011    1.7175
1100    1.5700
1101    1.3915
1110    1.3470
1111    2.7570
dtype: float64

In [28]:
plot_simulation(simulate([ZOX_CXM_avg], carrying_cap=10**9))

In [27]:
pairs[0][1]

Genotype
0000    1.879
0001    2.533
0010    0.133
0011    0.094
0100    2.456
0101    2.437
0110    0.083
0111    0.094
1000    2.198
1001    2.570
1010    2.308
1011    2.886
1100    2.504
1101    3.002
1110    2.528
1111    3.453
Name: SAM, dtype: float64

## Pure Python simulator

In [5]:
num_loci = 4
num_genotypes = 1 << num_loci

paths = [[0]]
def all_paths(r, current_path=[0]):
    current_genotype = current_path[-1]
    mutations = [current_genotype ^ (1 << i) for i in range(num_loci)]
    beneficial_mutations = [m for m in mutations if r[m] > r[current_genotype]]
    potential_paths = [current_path + [m] for m in beneficial_mutations]
    for p in potential_paths:
        if p not in paths:
            paths.append(p)
            all_paths(r, p)
#all_paths(r)
#[["{0:0>4b}".format(n) for n in path] for path in paths]

def greedy_path(r, seed=0):
    greedy_path = (seed,)
    current_genotype = seed
    while True:
        mutations = [current_genotype ^ (1 << i) for i in range(num_loci)]
        better_genotype = False
        for mut in mutations:
            if r[mut] > r[current_genotype]:
                better_genotype = True
                current_genotype = mut
        if better_genotype:
            greedy_path += (current_genotype,)
            continue
        else:
            break
    return greedy_path

def local_optima(r):
    optima = []
    for genotype, rate in enumerate(r):
        mutations = [genotype ^ (1 << i) for i in range(num_loci)]
        better_genotype = False
        for mut in mutations:
            if r[mut] > rate:
                better_genotype = True
        if not better_genotype:
            optima.append(genotype)
    optima.remove(np.argmax(r))
    return optima

def remove_regressions(N):
    paths_to_delete = []
    paths_to_add = {}
    for path, n in N.items():
        new_path = ()
        i = 0
        while i < len(path):
            g1 = path[i]
            new_path += (g1,)
            for j, g2 in enumerate(path[i+1:]):
                if g1 == g2:
                    i = i+j+1
            i += 1
        if new_path != path:
            paths_to_delete.append(path)
            if new_path not in paths_to_add:
                paths_to_add[new_path] = n
            else:
                paths_to_add[new_path] += n
    for p in paths_to_delete:
        del N[p]
    for p, n in paths_to_add.items():
        if p not in N:
            N[p] = n
        else:
            N[p] += n
    return(N)

def remove_zeros(N):
    paths_to_delete = []
    for path, n in N.items():
        if n == 0:
            paths_to_delete.append(path)
    for p in paths_to_delete:
        del N[p]
    return(N)

#pure python sim

num_loci = 4
num_genotypes = 1 << num_loci

def simulate_py(landscape, K = 10**9, P = 10**-8, seed_genotype = 0, simulation_length = 1200, frequency=None):
    try:
        landscape = [i if isinstance(i, list) else i.tolist() for i in landscape]
    except (TypeError, AttributeError) as err:
        raise TypeError('invalid landscape') from err
    r = landscape[0]
    r_num = 0
    N = {(seed_genotype,): K}
    first_appearances = {(seed_genotype,): (0,)}
    trace = []
    trace.append([K if i == seed_genotype else 0 for i in range(num_genotypes)])
    optimal_genotype = np.argmax(r)
    critical_times = [None, None, None]
    for t in range(1, simulation_length+1):
        # growth
        for path, n in N.items():
            fractpart, intpart = math.modf(n * math.exp(r[path[-1]]))
            if np.random.random() < fractpart:
                intpart += 1
            N[path] = int(intpart)
        # mutation
        mutants = [[0, []] for i in range(num_genotypes)]
        for path, n in N.items():
            if n == 0:
                continue
            mutants[path[-1]][0] += n
            mutants[path[-1]][1].append(path)
        for i, (n, paths) in enumerate(mutants):
            if n == 0:
                continue
            mu = n * P
            n = np.random.poisson(mu) # expected number of mutants
            for j in range(n):
                mutant_path = paths[np.random.randint(len(paths))]
                mutant_genotype = mutant_path[-1]
                locus = np.random.randint(num_loci)
                mutated_genotype = mutant_genotype ^ (1 << locus);
                mutated_path = (*mutant_path, mutated_genotype)
                if mutated_path in N:
                    N[mutated_path] += 1
                else:
                    N[mutated_path] = 1
                    first_appearances[mutated_path] = first_appearances[mutant_path] + (t,)
                N[mutant_path] -= 1
                if N[mutant_path] == 0:
                    paths.remove(mutant_path)
        # death
        sigmaN = sum(N.values())
        if sigmaN == 0:
            break
        for path, n in N.items():
            fractpart, intpart = math.modf((n / sigmaN) * K)
            if np.random.random() < fractpart:
                intpart += 1
            N[path] = int(intpart)
        # record-keeping
        row = [0]*num_genotypes
        for path, n in N.items():
            row[path[-1]] += n
        trace.append(row)
        for i, multiplier in enumerate([0, 0.5, 0.99]):
            if critical_times[i] is not None:
                continue
            if row[optimal_genotype] > multiplier * K:
                critical_times[i] = t
        # dynamic landscapes
        if frequency and t % frequency == 0:
            if r_num != len(landscape) - 1:
                r_num += 1
            else:
                r_num = 0
            r = landscape[r_num]
    # clean-up
    #print(len(N))
    N = remove_zeros(N)
    N = remove_regressions(N)
    #N = {tuple(["{0:0>4b}".format(n) for n in k]):v for (k, v) in N.items()}
    return {'N': N,'trace': trace, 'opt': optimal_genotype, 'crit': critical_times}

In [108]:
simulate_py(pairs[2], frequency=100)

{'N': {(0, 2, 3, 7): 999999977,
  (0, 2, 3, 7, 5): 5,
  (0, 2, 3, 7, 15): 7,
  (0, 2, 3, 7, 6): 9,
  (0, 2, 3): 3},
 'trace': [[1000000000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [999999986, 4, 3, 0, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
  [999999965, 5, 10, 0, 12, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
  [999999922, 8, 31, 0, 27, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
  [999999824, 8, 94, 0, 56, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0],
  [999999577, 10, 277, 0, 115, 0, 0, 0, 21, 0, 0, 0, 0, 0, 0, 0],
  [999998914, 11, 816, 0, 235, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0],
  [999997083, 11, 2397, 0, 478, 0, 0, 0, 32, 0, 0, 0, 0, 0, 0, 0],
  [999991946, 12, 7033, 0, 972, 0, 0, 0, 37, 0, 0, 0, 0, 0, 0, 0],
  [999977344, 11, 20631, 0, 1970, 0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0],
  [999935437, 13, 60509, 0, 3990, 0, 0, 0, 51, 0, 0, 0, 0, 0, 0, 0],
  [999814401, 13, 177450, 0, 8078, 0, 0, 0, 58, 0, 0, 0, 0, 0, 0, 0],
  [999463309, 12, 520267, 0, 16344, 0, 0, 0, 67, 0, 0, 0, 0, 0, 0, 0],
  [998442518, 11, 152

In [6]:
def pathway_analysis2(landscape, K=10**9, num=100, **kwargs):
    paths = {}
    for i in range(num):
        sim = simulate_py(landscape, K=K, **kwargs)
        for path, count in sim['N'].items():
            if count < 100:
                continue
            if path in paths:
                paths[path] += count
            else:
                paths[path] = count
    paths = {k: v / num for k,v in paths.items()}
    return paths

In [144]:
pathway_analysis2(pairs[2], frequency=25)

{(0, 4, 6, 7): 456244338.52,
 (0, 2, 6, 7): 542511319.21,
 (0, 4): 692297.64,
 (0, 7): 79238.73,
 (0, 2, 6): 472782.73}

In [146]:
pathway_analysis2(pairs[2], frequency=10, num=50)

{(0, 4, 6, 7): 628686533.44, (0, 2, 6, 7): 370944394.54, (0, 4): 369047.08}

In [147]:
pathway_analysis2(pairs[2], frequency=1, num=50)

{(0, 4, 6, 7): 975589402.74,
 (0, 7): 19856668.82,
 (0, 4): 83094.4,
 (0, 2, 6, 7): 4469532.1,
 (0, 2, 6): 1274.76}

In [143]:
pathway_analysis2([ZOX_CXM_avg])

{(0, 4, 6, 7): 948626502.45, (0, 2, 6, 7): 51373131.08, (0, 4): 337.8}

In [145]:
pathway_analysis2(pairs[2], frequency=50, num=50)

{(0, 2, 3, 7): 999998374.82, (0, 2): 1600.86}

In [141]:
simulate_py([ZOX_CXM_avg])

{'N': {(0, 4, 6, 7): 790365779,
  (0, 2, 6, 7): 209634192,
  (0, 4, 6, 7, 5): 2,
  (0, 2, 6, 7, 3): 3,
  (0, 2, 6, 7, 5): 2,
  (0, 4, 6, 7, 3): 2,
  (0, 4, 6, 7, 15): 5,
  (0, 2, 6, 7, 15): 6,
  (0, 4, 6): 3,
  (0, 2, 6): 4},
 'trace': [[1000000000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [999999990, 3, 1, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
  [999999976, 6, 5, 0, 9, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
  [999999948, 8, 13, 0, 26, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
  [999999886, 11, 29, 0, 70, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0],
  [999999739, 12, 60, 0, 183, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
  [999999382, 14, 123, 0, 474, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0],
  [999998504, 15, 249, 0, 1226, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0],
  [999996309, 15, 503, 0, 3168, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
  [999990781, 15, 1015, 0, 8183, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
  [999976805, 16, 2043, 0, 21129, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0],
  [999941313, 17, 4112, 0, 54552, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0,

In [44]:
# 50 and above take path 0, 2, 3, 7
plot_simulation(simulate(pairs[2], frequency=50))

In [51]:
# 25 and below take path 0, 4, 6, 7
plot_simulation(simulate(pairs[2], frequency=5), genotypes_to_plot=(0,4,6,7))

In [50]:
# the average also takes path 0, 4, 6, 7
plot_simulation(simulate([ZOX_CXM_avg]))

In [82]:
def box_switching_zox(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], k=9, num=100, analyses=None, mean=True, **kargs):
    global figin
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    data = []
    wx = []
    wy = []
    shapes = []
    param = dict(kargs)
    param['carrying_cap'] = 10**k
    if analyses:
        frequencies.append('avg')
    for i, f in enumerate(frequencies):
        param['frequency'] = f
        if analyses:
            analysis=analyses[f]
        else:
            analysis = pathway_analysis(landscapes, num=num, **param)
        times = [t for t in analysis['times'] if t != -1]
        trace = dict(
            type = 'box',
            #x = [f for i in range(len(times))],
            y = times,
            name = '&#120591;={}'.format(f) if f != 'avg' else f,
            boxpoints = False,
            line = dict(width=4)
        )
        data.append(trace)
        wx.append('&#120591;={}'.format(f) if f != 'avg' else f)
        wy.append(analysis['success_count'] / num)
        if mean and times:
            shapes.append(dict(
                type = 'line',
                xref = 'x',
                yref = 'y',
                x0 = i - 0.4,
                x1 = i + 0.4,
                y0 = np.mean(times),
                y1 = np.mean(times),
                line = {'color': DEFAULT_PLOTLY_COLORS[i % 10], 'width': 2}
                #layer = 'below'
            ))
    data.append(go.Scatter(
        x=wx,
        y=wy,
        name = 'Success rate',
        yaxis = 'y2',
        marker = dict(
            color = 'rgb(0, 0, 0)'
        ),
        mode='markers',
        cliponaxis = False
    ))
    layout = go.Layout(
        title = 'Running {} simulations on {} at different periods, K=10<sup>{}</sup>'.format(num, name, k),
        xaxis = dict(
            title = 'Switching period',
            tickvals = ['&#120591;={}'.format(f) if f != 'avg' else f for f in frequencies],
            #tickfont=dict(size=10),
            ticks='outside',
            showline=True,
            mirror=True
        ),
        yaxis = dict(
            title = 'Time to fixation',
            range=[0,250],
            tickvals=[0,25,50,75,100,125,150,175,200,225,250],
            ticktext=[0,'',50,'',100,'',150,'',200,'',250],
            showline=True,
            mirror=True
        ),
        yaxis2 = dict(
            title = 'Success rate',
            overlaying = 'y',
            scaleanchor='y',
            scaleratio=250,
            side = 'right',
            range=[0,1.0],
            tickvals=[0,0.2,0.4,0.6,0.8,1.0],
            zeroline=False
        ),
        showlegend = False,
        shapes = shapes,
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [83]:
frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1]
zox_analysis = {f : pathway_analysis(pairs[2], num=100, carrying_cap=10**9, frequency=f) for f in frequencies}

In [84]:
zox_analysis['avg'] = pathway_analysis([ZOX_CXM_avg], num=100, carrying_cap=10**9)

In [85]:
fig = box_switching_zox(pairs[2], analyses=zox_analysis, mean=False)
iplot(fig, show_link=False)
pio.write_image(fig, '2020-07/{}.pdf'.format('+'.join([ls.name for ls in pairs[2]])), scale=2)

In [8]:
CTX_SAM_avg = (pairs[0][0] + pairs[0][1]) / 2
CTX_SAM_avg

Genotype
0000    1.0195
0001    1.3090
0010    1.0345
0011    1.2210
0100    2.0545
0101    1.2875
0110    1.1890
0111    1.1815
1000    1.1915
1001    1.3550
1010    2.1385
1011    1.5445
1100    1.3645
1101    1.5470
1110    1.3235
1111    2.9325
dtype: float64

In [156]:
plot_simulation(simulate([CTX_SAM_avg]))

In [78]:
def box_switching_ctx(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], k=9, num=100, analyses=None, mean=True, **kargs):
    global figin
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    data = []
    wx = []
    wy = []
    shapes = []
    param = dict(kargs)
    param['carrying_cap'] = 10**k
    if analyses:
        frequencies.append('avg')
    for i, f in enumerate(frequencies):
        param['frequency'] = f
        if analyses:
            analysis=analyses[f]
        else:
            analysis = pathway_analysis(landscapes, num=num, **param)
        times = [t for t in analysis['times'] if t != -1]
        trace = dict(
            type = 'box',
            #x = [f for i in range(len(times))],
            y = times,
            name = '&#120591;={}'.format(f) if f != 'avg' else f,
            boxpoints = False,
            line = dict(width=4)
        )
        data.append(trace)
        wx.append('&#120591;={}'.format(f) if f != 'avg' else f)
        wy.append(analysis['success_count'] / num)
        if mean and times:
            shapes.append(dict(
                type = 'line',
                xref = 'x',
                yref = 'y',
                x0 = i - 0.4,
                x1 = i + 0.4,
                y0 = np.mean(times),
                y1 = np.mean(times),
                line = {'color': DEFAULT_PLOTLY_COLORS[i % 10], 'width': 2}
                #layer = 'below'
            ))
    data.append(go.Scatter(
        x=wx,
        y=wy,
        name = 'Success rate',
        yaxis = 'y2',
        marker = dict(
            color = 'rgb(0, 0, 0)'
        ),
        mode='markers',
        cliponaxis = False
    ))
    layout = go.Layout(
        title = 'Running {} simulations on {} at different periods, K=10<sup>{}</sup>'.format(num, name, k),
        xaxis = dict(
            title = 'Switching period',
            tickvals = ['&#120591;={}'.format(f) if f != 'avg' else f for f in frequencies],
            #tickfont=dict(size=10),
            ticks='outside',
            showline=True,
            mirror=True
        ),
        yaxis = dict(
            title = 'Time to fixation',
            range=[0,1500],
            tickvals=[0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500],
            ticktext=[0,'','','','',500,'','','','',1000,'','','','',1500],
            showline=True,
            mirror=True
        ),
        yaxis2 = dict(
            title = 'Success rate',
            overlaying = 'y',
            scaleanchor='y',
            scaleratio=1500,
            side = 'right',
            range=[0,1.0],
            tickvals=[0,0.2,0.4,0.6,0.8,1.0],
            zeroline=False
        ),
        showlegend = False,
        shapes = shapes,
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [75]:
frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1]
ctx_analysis = {f : pathway_analysis(pairs[0], num=100, carrying_cap=10**9, frequency=f) for f in frequencies}

In [76]:
ctx_analysis['avg'] = pathway_analysis([CTX_SAM_avg], num=100, carrying_cap=10**9)

In [81]:
fig = box_switching_ctx(pairs[0], analyses=ctx_analysis, mean=False)
iplot(fig, show_link=False)
pio.write_image(fig, '2020-07/{}.pdf'.format('+'.join([ls.name for ls in pairs[0]])), scale=2)

In [74]:
plot_simulation(simulate(pairs[0], frequency=10))

In [86]:
plot_simulation(simulate(pairs[0], frequency=75))

In [71]:
simulate_py(pairs[0], frequency=50)

{'N': {(0, 2, 3, 11, 15): 36038390,
  (0, 2, 10, 11, 15): 963961590,
  (0, 2, 3, 11, 15, 14): 1,
  (0, 2, 3, 11, 15, 13): 2,
  (0, 2, 10, 11, 15, 13): 5,
  (0, 2, 3, 11, 15, 7): 1,
  (0, 2, 10, 11, 15, 7): 1,
  (0, 2, 10, 11, 15, 14): 3,
  (0, 2, 10, 11): 5,
  (0, 2, 3, 11): 2},
 'trace': [[1000000000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [999999987, 3, 4, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0],
  [999999959, 4, 26, 0, 2, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0],
  [999999809, 6, 158, 0, 13, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0],
  [999998984, 8, 934, 0, 59, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0],
  [999994181, 11, 5520, 0, 266, 0, 0, 0, 22, 0, 0, 0, 0, 0, 0, 0],
  [999966173, 13, 32604, 0, 1186, 0, 0, 0, 23, 0, 0, 0, 0, 0, 0, 0],
  [999802143, 15, 192537, 0, 5279, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0],
  [998840418, 15, 1136066, 0, 23473, 0, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0],
  [993223986, 15, 6672090, 0, 103880, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0],
  [961408162, 15, 38144288, 0, 447502, 0, 0, 0, 33

In [13]:
pathway_analysis2(pairs[0], frequency=25, num=10)

{(0, 2, 10, 11, 15): 499999990.4,
 (0, 2, 10): 494126821.1,
 (0, 2, 10, 14): 1627.7,
 (0, 2, 10, 11): 5871536.2}

In [70]:
plot_simulation(simulate(pairs[0], frequency=50), genotypes_to_plot=(0, 2, 3, 4, 10, 11, 15))

In [61]:
plot_simulation(simulate(pairs[0], frequency=100))

In [18]:
simulate_py(pairs[0], frequency=50)

{'N': {(0, 2, 10, 11, 15): 999999983,
  (0, 2, 10, 11, 15, 7): 3,
  (0, 2, 10, 11, 15, 13): 6,
  (0, 2, 10, 11, 15, 14): 4,
  (0, 2, 10, 11): 5},
 'trace': [[1000000000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [999999987, 3, 1, 0, 6, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
  [999999953, 7, 10, 0, 28, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
  [999999803, 8, 61, 0, 126, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
  [999999057, 11, 362, 0, 563, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
  [999995327, 15, 2140, 0, 2509, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0],
  [999976165, 16, 12641, 0, 11169, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0],
  [999875605, 22, 74657, 0, 49705, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
  [999338164, 21, 440705, 0, 221093, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0],
  [996423572, 20, 2595300, 0, 981091, 0, 0, 0, 17, 0, 0, 0, 0, 0, 0, 0],
  [980617768, 19, 15085176, 0, 4297018, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0],
  [900610084, 16, 81826587, 0, 17563296, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0],
  [615988268, 12, 330549933, 0, 

In [20]:
pathway_analysis2(pairs[0], frequency=50, num=10)

{(0, 2, 10, 11, 15): 550533512.9, (0, 2, 3, 11, 15): 449466468.1}

In [28]:
simulate_py(pairs[0], frequency=20)

{'N': {(0, 2, 10): 999478470,
  (0, 2, 10, 11): 520278,
  (0, 2, 10, 8): 22,
  (0, 2, 10, 14): 1229,
  (0, 2): 2},
 'trace': [[1000000000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [999999994, 1, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [999999968, 4, 7, 0, 20, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [999999859, 5, 42, 0, 92, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
  [999999327, 6, 250, 0, 413, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0],
  [999996667, 9, 1477, 0, 1842, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
  [999983060, 10, 8724, 0, 8199, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0],
  [999911970, 11, 51522, 0, 36488, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0],
  [999533468, 12, 304184, 0, 162328, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0],
  [997486134, 15, 1792888, 0, 720950, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0],
  [986356252, 15, 10470977, 0, 3172739, 0, 0, 0, 17, 0, 0, 0, 0, 0, 0, 0],
  [928492719, 13, 58215539, 0, 13291707, 0, 0, 0, 20, 0, 1, 0, 0, 0, 0, 0],
  [697339758, 11, 258233063, 2, 44427141, 0, 0, 0, 19, 0, 6, 0, 0, 0, 0

In [39]:
pathway_analysis2(pairs[0], frequency=75, num=10)

{(0, 2, 3, 11, 15): 997829907.6, (0, 4, 6, 14, 15): 2170073.2}

In [40]:
pathway_analysis2(pairs[0], frequency=150, num=10)

{(0, 2, 3, 11, 15): 999999980.0}

In [57]:
pathway_analysis2(pairs[0], frequency=15, num=10)

{(0, 2, 10): 899970775.2,
 (0, 2, 10, 11): 28875.9,
 (0, 2, 10, 14): 329.7,
 (0, 4): 99999989.7}